# **LP4_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 8.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
G

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets.

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs)."""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

1. Xw = Amount of land (in acres) dedicated to wheat farming
2. Xc = Amount of land (in acres) dedicated to corn farming
3. Xb = Amount of land (in acres) dedicated to sugar beet farming

4. Pw = Production of wheat (in tons)
5. Pc = Production of corn (in tons)
6. Pb = Production of sugar beet (in tons)

7. Qw = Quantity of wheat (in tons) bought from wholesaler (if needed)
8. Qc = Quantity of corn (in tons) bought from wholesaler (if needed)

9. Sw = Selling price for wheat per ton ($)
10. Sc = Selling price for corn per ton ($)
11. Sb1 = Selling price for sugar beet per ton ($) within the quota
12. Sb2 = Selling price for sugar beet per ton ($) exceeding the quota

13. Cw = Cost to plant an acre of wheat ($)
14. Cc = Cost to plant an acre of corn ($)
15. Cb = Cost to plant an acre of sugar beet ($) 

16. Uw = Unit yield of wheat (in tons/acre)
17. Uc = Unit yield of corn (in tons/acre)
18. Ub = Unit yield of sugar beet (in tons/acre)

19. T = Land area available (in acres)

20. Bw =

In [17]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [13]:
print(response2.choices[0].message.content)

The objective function to minimize total costs, in this case, would be:

min Z = Cw*Xw + Cc*Xc + Cb*Xb + 1.4*Sw*Qw + 1.4*Sc*Qc - Sw*max(0, Uw*Xw - Bw) - Sc*max(0, Uc*Xc - Bc) - Sb1*min(Ub*Xb, Qb) - Sb2*max(0, Ub*Xb - Qb)


In [10]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [15]:
print(response3.choices[0].message.content)

The constraints for this problem are:

1. Land use Constraint:
Xw + Xc + Xb <= T

2. Minimum Requirements for Cattle Feed:
Uw*Xw + Qw >= Bw
Uc*Xc + Qc >= Bc

3. Constraints on Yield:
Pw = Uw*Xw
Pc = Uc*Xc
Pb = Ub*Xb

4. Constraints on Quantity of Wheat and Corn Bought from Wholesaler:
Qw >= 0
Qc >= 0

5. Non-negativity Constraints:
Xw, Xc, Xb, Qw, Qc >= 0

6. Integral Constraints:
Xw, Xc, Xb must be integers (since number of acres cannot be in decimal points) 

7. Constraints on Production of Sugar Beets:
Pb <= Qb + (Pb - Qb)/Sb2

The first two constraints ensure that the farmer does not use more land than what she has available and that she produces or purchases enough wheat and corn for her cattle. The next three constraints ensure that the yields are correctly calculated and the quantity of wheat and corn bought from the wholesaler is non-negative. The non-negativity constraints ensure that the amounts of land devoted to each crop is non-negative. Similarly, the integral constraints

### **Generate Pyomo Code**

In [16]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [18]:
Markdown(response4.choices[0].message.content)

Here is the Python Pyomo code for the optimization problem:

```python
# import pyomo.environ
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# define the decision variables
model.Xw = Var(domain=NonNegativeIntegers)
model.Xc = Var(domain=NonNegativeIntegers)
model.Xb = Var(domain=NonNegativeIntegers)
model.Qw = Var(domain=NonNegativeReals)
model.Qc = Var(domain=NonNegativeReals)

# parameters (assigning some random values for now)
model.Cw = Param(initialize=10)      # Cost to plant an acre of wheat ($)
model.Cc = Param(initialize=10)      # Cost to plant an acre of corn ($)
model.Cb = Param(initialize=10)      # Cost to plant an acre of sugar beet ($) 

model.Uw = Param(initialize=1)       # Unit yield of wheat (in tons/acre)
model.Uc = Param(initialize=1)       # Unit yield of corn (in tons/acre)
model.Ub = Param(initialize=1)       # Unit yield of sugar beet (in tons/acre)

model.T = Param(initialize=100)      # Area of available land (in acres)

model.Bw = Param(initialize=50)      # Minimum requirement of wheat for cattle feed (in tons)
model.Bc = Param(initialize=50)      # Minimum requirement of corn for cattle feed (in tons)

model.Qb = Param(initialize=20)      # Quota for Sugar beet (in tons)

model.Sw = Param(initialize=2)       # Selling price for wheat per ton ($)
model.Sc = Param(initialize=2)       # Selling price for corn per ton ($)
model.Sb1 = Param(initialize=2)      # Selling price for sugar beet per ton ($) within the quota
model.Sb2 = Param(initialize=1)      # Selling price for sugar beet per ton ($) exceeding the quota

# objective function
model.obj = Objective(expr = model.Cw*model.Xw + model.Cc*model.Xc + model.Cb*model.Xb + 1.4*model.Sw*model.Qw + 1.4*model.Sc*model.Qc
                      - model.Sw*max(0,model.Uw*model.Xw - model.Bw)
                      - model.Sc*max(0,model.Uc*model.Xc - model.Bc)
                      - model.Sb1*min(model.Ub*model.Xb,model.Qb)
                      - model.Sb2*max(0,model.Ub*model.Xb - model.Qb),
                      sense = minimize)

# constraints
model.land = Constraint(expr = model.Xw + model.Xc + model.Xb <= model.T)
model.req_wheat = Constraint(expr = model.Uw*model.Xw + model.Qw >= model.Bw)
model.req_corn = Constraint(expr = model.Uc*model.Xc + model.Qc >= model.Bc)

# solve
solver = SolverFactory('glpk')
solver.solve(model)

# print the optimal solution
print("The optimal solution is")
print("Xw = ", model.Xw())
print("Xc = ", model.Xc())
print("Xb = ", model.Xb())
print("Qw = ", model.Qw())
print("Qc = ", model.Qc())
```

Make sure to substitute the appropriate values you wish in the places where we have used example data. The values for `initialize` in Param are example values. The example values have been randomly chosen and do not represent real-world values. The constraints show that we should not exceed the total available land and the minimum requirements for wheat and corn for cattle feed should be satisfied. The objective function shows that we aim to minimize cost. Please install the glpk solver in your environment before running the script.

### **Run the code Generated by GPT4**

In [19]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# define the decision variables
model.Xw = Var(domain=NonNegativeIntegers)
model.Xc = Var(domain=NonNegativeIntegers)
model.Xb = Var(domain=NonNegativeIntegers)
model.Qw = Var(domain=NonNegativeReals)
model.Qc = Var(domain=NonNegativeReals)

# parameters (assigning some random values for now)
model.Cw = Param(initialize=10)      # Cost to plant an acre of wheat ($)
model.Cc = Param(initialize=10)      # Cost to plant an acre of corn ($)
model.Cb = Param(initialize=10)      # Cost to plant an acre of sugar beet ($)

model.Uw = Param(initialize=1)       # Unit yield of wheat (in tons/acre)
model.Uc = Param(initialize=1)       # Unit yield of corn (in tons/acre)
model.Ub = Param(initialize=1)       # Unit yield of sugar beet (in tons/acre)

model.T = Param(initialize=100)      # Area of available land (in acres)

model.Bw = Param(initialize=50)      # Minimum requirement of wheat for cattle feed (in tons)
model.Bc = Param(initialize=50)      # Minimum requirement of corn for cattle feed (in tons)

model.Qb = Param(initialize=20)      # Quota for Sugar beet (in tons)

model.Sw = Param(initialize=2)       # Selling price for wheat per ton ($)
model.Sc = Param(initialize=2)       # Selling price for corn per ton ($)
model.Sb1 = Param(initialize=2)      # Selling price for sugar beet per ton ($) within the quota
model.Sb2 = Param(initialize=1)      # Selling price for sugar beet per ton ($) exceeding the quota

# objective function
model.obj = Objective(expr = model.Cw*model.Xw + model.Cc*model.Xc + model.Cb*model.Xb + 1.4*model.Sw*model.Qw + 1.4*model.Sc*model.Qc
                      - model.Sw*max(0,model.Uw*model.Xw - model.Bw)
                      - model.Sc*max(0,model.Uc*model.Xc - model.Bc)
                      - model.Sb1*min(model.Ub*model.Xb,model.Qb)
                      - model.Sb2*max(0,model.Ub*model.Xb - model.Qb),
                      sense = minimize)

# constraints
model.land = Constraint(expr = model.Xw + model.Xc + model.Xb <= model.T)
model.req_wheat = Constraint(expr = model.Uw*model.Xw + model.Qw >= model.Bw)
model.req_corn = Constraint(expr = model.Uc*model.Xc + model.Qc >= model.Bc)

# solve
solver = SolverFactory('glpk')
solver.solve(model)

# print the optimal solution
print("The optimal solution is")
print("Xw = ", model.Xw())
print("Xc = ", model.Xc())
print("Xb = ", model.Xb())
print("Qw = ", model.Qw())
print("Qc = ", model.Qc())

PyomoException: Cannot convert non-constant Pyomo expression (0  <  Xw - 50) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [25]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# define the decision variables
model.Xw = Var(domain=NonNegativeIntegers)
model.Xc = Var(domain=NonNegativeIntegers)
model.Xb = Var(domain=NonNegativeIntegers)
model.Qw = Var(domain=NonNegativeReals)
model.Qc = Var(domain=NonNegativeReals)

# parameters (assigning some random values for now)
model.Cw = Param(initialize=10)      # Cost to plant an acre of wheat ($)
model.Cc = Param(initialize=10)      # Cost to plant an acre of corn ($)
model.Cb = Param(initialize=10)      # Cost to plant an acre of sugar beet ($)

model.Uw = Param(initialize=1)       # Unit yield of wheat (in tons/acre)
model.Uc = Param(initialize=1)       # Unit yield of corn (in tons/acre)
model.Ub = Param(initialize=1)       # Unit yield of sugar beet (in tons/acre)

model.T = Param(initialize=100)      # Area of available land (in acres)

model.Bw = Param(initialize=50)      # Minimum requirement of wheat for cattle feed (in tons)
model.Bc = Param(initialize=50)      # Minimum requirement of corn for cattle feed (in tons)

model.Qb = Param(initialize=20)      # Quota for Sugar beet (in tons)

model.Sw = Param(initialize=2)       # Selling price for wheat per ton ($)
model.Sc = Param(initialize=2)       # Selling price for corn per ton ($)
model.Sb1 = Param(initialize=2)      # Selling price for sugar beet per ton ($) within the quota
model.Sb2 = Param(initialize=1)      # Selling price for sugar beet per ton ($) exceeding the quota

model.b1 = Var(domain=Binary)
model.b2 = Var(domain=Binary)
model.b3 = Var(domain=Binary)
model.b4 = Var(domain=Binary)


M = 100000000

model.Yb = Var(domain=NonNegativeReals) #aux variable

# objective function
model.obj = Objective(expr = model.Cw*model.Xw + model.Cc*model.Xc + model.Cb*model.Xb + 1.4*model.Sw*model.Qw + 1.4*model.Sc*model.Qc - model.Sw*(model.Uw*model.Xw - model.Bw) - model.Sc*(model.Uc*model.Xc - model.Bc) - model.Sb1*model.Yb - model.Sb2*(model.Ub*model.Xb - model.Qb),sense = minimize)

# constraints
model.land = Constraint(expr = model.Xw + model.Xc + model.Xb <= model.T)
model.req_wheat = Constraint(expr = model.Uw*model.Xw + model.Qw >= model.Bw)
model.req_corn = Constraint(expr = model.Uc*model.Xc + model.Qc >= model.Bc)


model.objconstraint1 = Constraint(expr= model.Uw*model.Xw - model.Bw <= model.b1 * M)
model.objconstraint2 = Constraint(expr= model.Uw*model.Xw - model.Bw <= M * (1 - model.b1))

model.objconstraint3 = Constraint(expr= model.Uc*model.Xc - model.Bc <= model.b2 * M)
model.objconstraint4 = Constraint(expr= model.Uc*model.Xc - model.Bc <= M * (1 - model.b2))

model.objconstraint5 = Constraint(expr= model.Uc*model.Xc - model.Bc <= model.b2 * M)
model.objconstraint6 = Constraint(expr= model.Uc*model.Xc - model.Bc <= M * (1 - model.b2))

model.objconstraint7 = Constraint(expr= model.Ub*model.Xb <= model.b3 * M)
model.objconstraint8 = Constraint(expr= model.Uc*model.Xc - model.Bc <= M * (1 - model.b2))

model.MinConstraint1 = Constraint(expr=model.Yb <= model.Ub * model.Xb)
model.MinConstraint2 = Constraint(expr=model.Yb <= model.Qb)

model.BigMConstraint1 = Constraint(expr=model.Yb >= model.Ub * model.Xb - M * (1 - model.b3))
model.BigMConstraint2 = Constraint(expr=model.Yb >= model.Qb - M * model.b3)

model.objconstraint9 = Constraint(expr= model.Ub*model.Xb - model.Qb <= model.b4 * M)
model.objconstraint10 = Constraint(expr= model.Ub*model.Xb - model.Qb <= M * (1 - model.b4))

model.sugar_cons = Constraint(expr= model.Uw*model.Xw <= model.Qb + (model.Uw*model.Xw - model.Qb)*(1/2))
# solve
solver = SolverFactory('glpk')
solver.solve(model)

# print the optimal solution
print("The optimal solution is")
print("Xw = ", model.Xw())
print("Xc = ", model.Xc())
print("Xb = ", model.Xb())
print("Qw = ", model.Qw())
print("Qc = ", model.Qc())
print(model.obj())

The optimal solution is
Xw =  0.0
Xc =  0.0
Xb =  0.0
Qw =  50.0
Qc =  50.0
500.0
